In [ ]:
#pip install tespy

In [1]:
# Não mexer nessa parte
from tespy.networks import network
from tespy.components import (
    turbine, splitter, merge, condenser, pump, sink, source,
    heat_exchanger_simple, desuperheater, cycle_closer
)
from tespy.connections import connection, bus
from tespy.tools import logger
import logging

import numpy as np


logger.define_logging(screen_level=logging.ERROR)


class PowerPlant():

    def __init__(self):
        self.nw = network(
            fluids=['BICUBIC::water'],
            p_unit='bar', T_unit='C', h_unit='kJ / kg',
            iterinfo=False)
        # components
        # main cycle
        eco = heat_exchanger_simple('economizer')
        eva = heat_exchanger_simple('evaporator')
        sup = heat_exchanger_simple('superheater')
        cc = cycle_closer('cycle closer')
        hpt = turbine('high pressure turbine')
        sp1 = splitter('splitter 1', num_out=2)
        mpt = turbine('mid pressure turbine')
        sp2 = splitter('splitter 2', num_out=2)
        lpt = turbine('low pressure turbine')
        con = condenser('condenser')
        pu1 = pump('feed water pump')
        fwh1 = condenser('feed water preheater 1')
        fwh2 = condenser('feed water preheater 2')
        dsh = desuperheater('desuperheater')
        me2 = merge('merge2', num_in=2)
        pu2 = pump('feed water pump 2')
        pu3 = pump('feed water pump 3')
        me = merge('merge', num_in=2)

        # cooling water
        cwi = source('cooling water source')
        cwo = sink('cooling water sink')

        # connections
        # main cycle
        cc_hpt = connection(cc, 'out1', hpt, 'in1', label='feed steam')
        hpt_sp1 = connection(hpt, 'out1', sp1, 'in1', label='extraction1')
        sp1_mpt = connection(sp1, 'out1', mpt, 'in1', state='g')
        mpt_sp2 = connection(mpt, 'out1', sp2, 'in1', label='extraction2')
        sp2_lpt = connection(sp2, 'out1', lpt, 'in1')
        lpt_con = connection(lpt, 'out1', con, 'in1')
        con_pu1 = connection(con, 'out1', pu1, 'in1')
        pu1_fwh1 = connection(pu1, 'out1', fwh1, 'in2')
        fwh1_me = connection(fwh1, 'out2', me, 'in1', state='l')
        me_fwh2 = connection(me, 'out1', fwh2, 'in2', state='l')
        fwh2_dsh = connection(fwh2, 'out2', dsh, 'in2', state='l')
        dsh_me2 = connection(dsh, 'out2', me2, 'in1')
        me2_eco = connection(me2, 'out1', eco, 'in1', state='l')
        eco_eva = connection(eco, 'out1', eva, 'in1')
        eva_sup = connection(eva, 'out1', sup, 'in1')
        sup_cc = connection(sup, 'out1', cc, 'in1')

        self.nw.add_conns(cc_hpt, hpt_sp1, sp1_mpt, mpt_sp2, sp2_lpt,
                          lpt_con, con_pu1, pu1_fwh1, fwh1_me, me_fwh2,
                          fwh2_dsh, dsh_me2, me2_eco, eco_eva, eva_sup, sup_cc)

        # cooling water
        cwi_con = connection(cwi, 'out1', con, 'in2')
        con_cwo = connection(con, 'out2', cwo, 'in1')

        self.nw.add_conns(cwi_con, con_cwo)

        # preheating
        sp1_dsh = connection(sp1, 'out2', dsh, 'in1')
        dsh_fwh2 = connection(dsh, 'out1', fwh2, 'in1')
        fwh2_pu2 = connection(fwh2, 'out1', pu2, 'in1')
        pu2_me2 = connection(pu2, 'out1', me2, 'in2')

        sp2_fwh1 = connection(sp2, 'out2', fwh1, 'in1')
        fwh1_pu3 = connection(fwh1, 'out1', pu3, 'in1')
        pu3_me = connection(pu3, 'out1', me, 'in2')

        self.nw.add_conns(sp1_dsh, dsh_fwh2, fwh2_pu2, pu2_me2,
                          sp2_fwh1, fwh1_pu3, pu3_me)

        # busses
        # power bus
        self.power = bus('power')
        self.power.add_comps(
            {'comp': hpt, 'char': -1}, {'comp': mpt, 'char': -1},
            {'comp': lpt, 'char': -1}, {'comp': pu1, 'char': -1},
            {'comp': pu2, 'char': -1}, {'comp': pu3, 'char': -1})

        # heating bus
        self.heat = bus('heat')
        self.heat.add_comps(
            {'comp': eco, 'char': 1}, {'comp': eva, 'char': 1},
            {'comp': sup, 'char': 1})

        self.nw.add_busses(self.power, self.heat)

        # parametrization
        # components
        hpt.set_attr(eta_s=0.9)
        mpt.set_attr(eta_s=0.9)
        lpt.set_attr(eta_s=0.9)

        pu1.set_attr(eta_s=0.8)
        pu2.set_attr(eta_s=0.8)
        pu3.set_attr(eta_s=0.8)

        eco.set_attr(pr=0.99)
        eva.set_attr(pr=0.99)
        sup.set_attr(pr=0.99)

        con.set_attr(pr1=0.99, pr2=0.99, ttd_u=5)
        fwh1.set_attr(pr1=0.99, pr2=0.99, ttd_u=5)
        fwh2.set_attr(pr1=0.99, pr2=0.99, ttd_u=5)
        dsh.set_attr(pr1=0.99, pr2=0.99)

        # connections
        eco_eva.set_attr(x=0)
        eva_sup.set_attr(x=1)

        cc_hpt.set_attr(m=200, T=650, p=100, fluid={'water': 1})
        hpt_sp1.set_attr(p=20)
        mpt_sp2.set_attr(p=3)
        lpt_con.set_attr(p=0.05)

        cwi_con.set_attr(T=20, p=10, fluid={'water': 1})

    def calculate_efficiency(self, x):
        # set extraction pressure
        self.nw.connections['extraction1'].set_attr(p=x[0])
        self.nw.connections['extraction2'].set_attr(p=x[1])
        #self.nw.connections['feed steam'].set_attr(m=x[2], T=x[3], p=x[4])

        self.nw.solve('design')

        #for cp in self.nw.components.values():
        #    if isinstance(cp, condenser) or isinstance(cp, desuperheater):
        #        if cp.Q.val > 0:
        #            return np.nan
        #    elif isinstance(cp, pump):
        #        if cp.P.val < 0:
        #            return np.nan
        #    elif isinstance(cp, turbine):
        #        if cp.P.val > 0:
        #            return np.nan

        if self.nw.res[-1] > 1e-3 or self.nw.lin_dep:
         #   return np.nan
            return self.nw.busses['power'].P.val / self.nw.busses['heat'].P.val
        else:

            return self.nw.busses['power'].P.val / self.nw.busses['heat'].P.val

In [2]:
# Não mexer nessa parte, só precisa mudar a função "def fitness(self, x):" quando necessário

import math
class optimization_problem():

    def fitness(self, x):
        f1 = 1/self.model.calculate_efficiency(x) # problema de minimização
        #f1 = self.model.calculate_efficiency(x) # problema de maximização
        if math.isnan(f1): # se encontrar um valor impossível, põe a pior eficiência
            f1 = 10.0 #  minimizar
           # f1 = 0.0 #  maximizar
        return f1

    def get_nobj(self):
        """Return number of objectives."""
        return 1

    # equality constraints
    def get_nec(self):
        return 0

    # inequality constraints
    def get_nic(self):
        return 1

    def get_bounds(self):
        """Return bounds of decision variables."""
        return ([1, 1], [40, 40])
    
optimize = optimization_problem()
optimize.model = PowerPlant()

In [3]:
# usa essa parte
x = [25.7083,2.6677] # essa aqui são as duas entradas
optimize.fitness(x)# aqui entra dois valores

2.229520246007255

In [4]:
# Gera um arquivo com todas as repetições e faz um gráfico com as médias 

import random
import math    
import copy   
import sys     
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np



#Hyperparameters
#topology = "star" # global, circle, star
name_topology = ['circle', 'global'] #['apso', 'global', 'circle', 'star'] # ['global'] - if it need only one topology, or use ['global', 'circle', 'star']
dim = 2 # max of 30 for sphere and rastringin
num_particles = 10
max_epochs = 20
min_velocity = -0.5
max_velocity = 0.5
w_variable = 2 # 0 - w is constant, 1 - w changes linearly, 2 - w uses Clerc Constrition
w_fixed = 0.8    # inertia value if it is fixed
w_max = 0.9      # inertia max value if it is variable
w_min = 0.4      # inertia min value if it is variable
c1 = 2.05 # cognitive (particle) - If it is been used Clerc constriction factor, c1+c2 must be major than 4
c2 = 2.05 # social (swarm)
r_fixed = 0 # 0 - r randomly defined for each interaction, 1 - r defined as fixed r1_def and r2def
r1_def = 0.25
r2_def = 0.25
error_function = 'power_plant' # 'sphere', 'rastrigin', 'ackley', 'rosenbrock', 'power_plant'
minx_bound = 0 #boundary of the function  sphere (−100, 100), Rastrigin (−5.12, 5.12), 'ackley' (-32, 32), 'rosenbrock' (-30, 30)
maxx_bound = 40 #boundary of the function
mult_show = 1 # reason of interactions number that the particle position will be shown
repeticoes = 5 # quantity of time that every test will be runned

# APSO
beta = 0.5 # 0.1 a 0.7
sigma_apso = 0.6 # 0.1 a 0.99

#If initial positions and speeds are been used
defined_init_posit = 0  # 0 posição randômica, 1 poisção definida pelos parametros init_pos
init_pos = [[5 , 5], [8 , 3], [6 , 7]]  # definir como uma matriz com as dimensões [num_particles][dim]

defined_init_vel = 0  # 0 posição randômica, 1 poisção definida pelos parametros init_vel
init_vel = [[2 , 2], [3 , 3], [4 , 4]]  # definir como uma matriz com as dimensões [num_particles][dim]

defined_int_best = 0 # 0 valores de melhores posição calculados, 1 valores de melhores posições definidos para a 1° interação
init_pos_best = [[5 , 5], [8 , 3], [6 , 7]]  # definir como uma matriz com as dimensões [num_particles][dim]
init_pos_global_best = [5 , 5] # definir como um vetor com dimensão [dim]

#Parameters
sigma = c1+c2 # Clerc constriction factor, it must be major than 4
file_save = pd.DataFrame()
file_temp = pd.Series([])
nome_error_function = error_function # sphere, rastrigin , ackley, rosenbrock
# ------------------------------------

def show_vector(vector):
  for i in range(len(vector)):
    if i % 8 == 0: # 8 columns
      print("\n", end="")
    if vector[i] >= 0.0:
      print(' ', end="")
    print("%.4f" % vector[i], end="") # 4 decimals
    print(" ", end="")
  print("\n")

def rastrigin(position):
  err = 0.0
  for i in range(len(position)):
    xi = position[i]
    err += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
  return err


def rosenbrock(position): # The Rosenbrock funtion goes always from 1 to n-1
  err = 0.0
  for i in range(1, len(position)-1):
    xi = position[i]
    xj = position[i+1]
    err += 100 * (xj - xi ** 2) ** 2 + (xi - 1) ** 2
  return err


def sphere(position):
  err = 0.0
  for i in range(len(position)):
    xi = position[i]
    err += (xi * xi)
  return err


def ackley(position):
  firstSum = 0.0
  secondSum = 0.0
  for i in range(len(position)):
    c = position[i]
    firstSum += c**2.0
    secondSum += math.cos(2.0*math.pi*c)
  n = float(len(position))
  return -20.0*math.exp(-0.2*math.sqrt(firstSum/n)) - math.exp(secondSum/n) + 20 + math.e

def power_plant(position):
    return optimize.fitness(position)



# ------------------------------------


class Particle:
  def __init__(self, dim, minx, maxx, seed):
    self.rnd = random.Random(datetime.now())
    self.position = [0.0 for i in range(dim)]
    self.velocity = [0.0 for i in range(dim)]
    self.best_part_pos = [0.0 for i in range(dim)]
    self.seed = seed
    
    if defined_init_posit == 0:
        for i in range(dim):
            self.position[i] = ((maxx_bound - minx_bound) * self.rnd.random() + minx_bound)        
    else:
        for i in range(dim):
            self.position[i] = init_pos[self.seed][i]
            
    if defined_init_vel == 0:
        for i in range(dim):
            self.velocity[i] = ((maxx - minx) * self.rnd.random() + minx)       
    else:
        for i in range(dim):
            self.velocity[i] = init_vel[self.seed][i]                  
            

    if error_function == 'sphere':
        self.error = sphere(self.position) # curr error
    if error_function == 'rastrigin':
        self.error = rastrigin(self.position) # curr error
    if error_function == 'ackley':
        self.error = ackley(self.position) # curr error
    if error_function == 'rosenbrock':
        self.error = rosenbrock(self.position) # curr error
    if error_function == 'power_plant':
        self.error = power_plant(self.position) # curr error
        
    self.best_part_pos = copy.copy(self.position) 
    self.best_part_err = self.error # best error

def Solve(max_epochs, n, dim, minx, maxx, topology):
#  rnd = random.Random(0)
  rnd = random.Random(datetime.now())

  # create n random particles
  swarm = [Particle(dim, minx, maxx, i) for i in range(n)] 

  best_swarm_pos = [0.0 for i in range(dim)] # not necess.
  best_swarm_err = sys.float_info.max # swarm best
  for i in range(n): # check each particle
    if swarm[i].error < best_swarm_err:
      best_swarm_err = swarm[i].error
      best_swarm_pos = copy.copy(swarm[i].position) 

  epoch = 0

  if defined_init_posit == 1:
    for i in range(n):
        swarm[i].best_part_pos = init_pos_best[i]
    best_swarm_pos = init_pos_global_best    

    
  while epoch < max_epochs:
    if w_variable == 1:  # w changes linearly
      w_int =  (w_max - w_min)/max_epochs
      w = w_max - epoch*w_int 
      k_constric = 1  
    elif w_variable == 0:  # w is constant
      w = w_fixed
      k_constric = 1
    elif w_variable == 2: # it uses Clerc constriction 
      w = 1
      k_constric = 2/abs(2 - sigma - (sigma*sigma - 4*sigma)**(1/2))
  #  elif w_variable == 3:  # it uses APSO
  #    w = 1
  #    k_constric = 1  
        
    
    if epoch % (mult_show) == 0 and epoch > 1:
      print("Epoch = " + str(epoch) +
        " best error = %.6f" % best_swarm_err)
      print(best_swarm_pos)
      #for j in range(n):
       # print("\n Particle %s : " % j)
       # show_vector(swarm[j].position)

    for i in range(n): # process each particle
      
      # compute new velocity of curr particle
      for k in range(dim): 
        if r_fixed == 1:
            r1 = r1_def
            r2 = r2_def
        else:
            r1 = rnd.random()    # randomizations
            r2 = rnd.random()
    
        if topology == "circle":
            best_erro_circle = swarm[i].best_part_err
            best_pos_circle = swarm[i].best_part_pos
            for r in range(2):    
                if swarm[(i-1)+r].best_part_err < best_erro_circle:
                    best_erro_circle = swarm[(i-1)+r].best_part_err
                    best_pos_circle = swarm[(i-1)+r].best_part_pos  
            
            swarm[i].velocity[k] = k_constric*( (w * swarm[i].velocity[k]) + 
                                    (c1 * r1 * (swarm[i].best_part_pos[k] - swarm[i].position[k])) +  
                                    (c2 * r2 * (best_pos_circle[k] - swarm[i].position[k])) )
            
        if topology == "global":
            swarm[i].velocity[k] = k_constric*( (w * swarm[i].velocity[k]) + 
                                    (c1 * r1 * (swarm[i].best_part_pos[k] - swarm[i].position[k])) +  
                                    (c2 * r2 * (best_swarm_pos[k] - swarm[i].position[k])) )
        
        if topology == "star": 
            lead_particle = 1    # define a particula 1 como o centro da topologia
            if i == lead_particle:  
                swarm[i].velocity[k] = k_constric*( (w * swarm[i].velocity[k]) + 
                                        (c1 * r1 * (swarm[i].best_part_pos[k] - swarm[i].position[k])) +  
                                        (c2 * r2 * (best_swarm_pos[k] - swarm[i].position[k])) )        
            else:
                swarm[i].velocity[k] = k_constric*( (w * swarm[i].velocity[k]) + 
                                        (c1 * r1 * (swarm[i].best_part_pos[k] - swarm[i].position[k])) +  
                                        (c2 * r2 * (swarm[lead_particle].best_part_pos[k] - swarm[i].position[k])) )
            

        if topology != "apso": 
            if swarm[i].velocity[k] < minx:
                swarm[i].velocity[k] = minx
            elif swarm[i].velocity[k] > maxx:
                swarm[i].velocity[k] = maxx


      # compute new position using new velocity
      alfa = sigma_apso**epoch
      for k in range(dim):
        if topology != "apso":     
            swarm[i].position[k] += swarm[i].velocity[k]
        if topology == "apso":    
            swarm[i].position[k] = (1-beta)*swarm[i].position[k] + beta*best_swarm_pos[k] + alfa*random.randint(0, 1)
            
        # Verify if the position is inside of the function boundery for each particle
        if swarm[i].position[k] < minx_bound:
            swarm[i].position[k] = minx_bound
        elif swarm[i].position[k] > maxx_bound:
            swarm[i].position[k] = maxx_bound

  
      # compute error of new position
      if error_function == 'sphere':
        swarm[i].error = sphere(swarm[i].position)
      if error_function == 'rastrigin':
        swarm[i].error = rastrigin(swarm[i].position)
      if error_function == 'ackley':
        swarm[i].error = ackley(swarm[i].position)
      if error_function == 'rosenbrock':
        swarm[i].error = rosenbrock(swarm[i].position)
      if error_function == 'power_plant':
        swarm[i].error = power_plant(swarm[i].position)
        
      # is new position a new best for the particle?
      if swarm[i].error < swarm[i].best_part_err:
        swarm[i].best_part_err = swarm[i].error
        swarm[i].best_part_pos = copy.copy(swarm[i].position)

      # is new position a new best overall?
      if swarm[i].error < best_swarm_err:
        best_swarm_err = swarm[i].error
        best_swarm_pos = copy.copy(swarm[i].position)
      
    # salva num arquivo
      if error_function == 'sphere':
        file_temp[epoch] = sphere(best_swarm_pos)
      if error_function == 'rastrigin':
        file_temp[epoch] = rastrigin(best_swarm_pos)
      if error_function == 'ackley':
        file_temp[epoch] = ackley(best_swarm_pos)
      if error_function == 'rosenbrock':
        file_temp[epoch] = rosenbrock(best_swarm_pos)
      if error_function == 'power_plant':
        file_temp[epoch] = power_plant(best_swarm_pos)
        
    # Atualize w if it is not constant
    epoch += 1

  #  file_save 
  file_save.insert(0, "error_%02d" %vezes, file_temp )
  # while 
  return best_swarm_pos
# end Solve






print("\nStarting PSO algorithm\n")
for top in name_topology:
  topology = top  
  print("Topologia " + str(top))
  for vezes in range(repeticoes):
    print("Repetição " + str(vezes))
    best_position = Solve(max_epochs, num_particles, dim, min_velocity, max_velocity, topology)
  file_save['mean'] = file_save.mean(axis=1)  
  file_save.to_csv('Function %s' %nome_error_function +' Topology %s' %topology +'.csv', index = True)  
  for vezes in range(repeticoes):
    del file_save["error_%02d" %vezes]


print("\nPSO completed\n")
print("\nBest solution found:")
show_vector(best_position)
if error_function == 'sphere':
  err = sphere(best_position)
if error_function == 'rastrigin':
  err = rastrigin(best_position)
if error_function == 'ackley':
  err = ackley(best_position)
if error_function == 'rosenbrock': 
  err = rosenbrock(best_position)
if error_function == 'power_plant': 
  err = power_plant(best_position)

print("Error of best solution = %.6f" % err)


# Criate Graphs
#input_csv_0 = pd.read_csv('Function %s' %nome_error_function + ' Topology apso.csv')
input_csv_1 = pd.read_csv('Function %s' %nome_error_function + ' Topology global.csv')
input_csv_2 = pd.read_csv('Function %s' %nome_error_function + ' Topology circle.csv')
#input_csv_3 = pd.read_csv('Function %s' %nome_error_function + ' Topology star.csv')

## line 0 points 
#x0 = input_csv_0["Unnamed: 0"]
#y0 = input_csv_0['mean']  
#plt.plot(x0, y0, label = "APSO") 

# line 1 points 
x1 = input_csv_1["Unnamed: 0"]
y1 = input_csv_1['mean']  
plt.plot(x1, y1, label = "Global") 
  
# line 2 points 
x2 = input_csv_2["Unnamed: 0"]
y2 = input_csv_2['mean'] 
plt.plot(x2, y2, label = "Circle") 
         
## line 3 points 
#x3 = input_csv_3["Unnamed: 0"]
#y3 = input_csv_3['mean'] 
#plt.plot(x3, y3, label = "Star")           

plt.xlabel('Epoch') 
plt.ylabel('Error') 
plt.title('Average Error Graph for Topology Function %s' %nome_error_function) 
plt.legend() 
plt.savefig('Average Error Graph for Topology Function %s' %nome_error_function + '.png')


Starting PSO algorithm

Topologia circle
Repetição 0
Epoch = 2 best error = 2.232008
[33.97851672034692, 3.8604255605191184]
Epoch = 3 best error = 2.231882
[34.01826848701371, 3.757694817102631]
Epoch = 4 best error = 2.231805
[34.04728106698259, 3.6827174221703]
Epoch = 5 best error = 2.231757
[34.06845571825046, 3.627995636228891]
Epoch = 6 best error = 2.231726
[34.0839099059441, 3.5880572806842643]
Epoch = 7 best error = 2.231706
[34.09518904883287, 3.558908519981957]
Epoch = 8 best error = 2.231693
[34.10342106120565, 3.5376344780517375]
Epoch = 9 best error = 2.231684
[34.109429144299725, 3.522107750700585]
Epoch = 10 best error = 2.231678
[34.11381410642449, 3.5107756651933832]
Epoch = 11 best error = 2.231673
[34.11701444379243, 3.502505012979413]
Epoch = 12 best error = 2.231670
[34.11935019014033, 3.4964687288372764]
Epoch = 13 best error = 2.231668
[34.12105492010299, 3.4920631843527605]
Epoch = 14 best error = 2.231667
[34.12229910667668, 3.4888478250774133]
Epoch = 15 be

KeyboardInterrupt: 

In [5]:
1/2.242833

0.4458646720464698